# Image Classification: How to run inference on the endpoint you have created?

### Download example images and class_id_to_label_mapping
The class_id_to_label_mapping file has been downloaded from [TensorFlow](https://storage.googleapis.com/download.tensorflow.org/data/ImageNetLabels.txt). [Apache 2.0 License](https://www.apache.org/licenses/LICENSE-2.0)

In [15]:
import boto3
from IPython.core.display import HTML

region = boto3.Session().region_name
s3_bucket = f"jumpstart-cache-prod-{region}"
key_prefix = "inference-notebook-assets"
s3 = boto3.client("s3")

def download_from_s3(key_filenames):
    for key_filename in key_filenames:
        s3.download_file(s3_bucket, f"{key_prefix}/{key_filename}", key_filename)

cat_jpg, dog_jpg, ImageNetLabels = "cat.jpg", "dog.jpg", "ImageNetLabels.txt"
download_from_s3(key_filenames=[cat_jpg, dog_jpg, ImageNetLabels])

HTML('<table><tr><td> <img src="cat.jpg" alt="cat" style="height: 250px;"/> <figcaption>cat.jpg</figcaption>'
     '</td><td> <img src="dog.jpg" alt="dog" style="height: 250px;"/> <figcaption>dog.jpg</figcaption>'
     '</td></tr></table>')

cat.jpg,dog.jpg


### Open the downloaded images and load in memory. You can upload any image from your local computer in the directory and open them here.

In [16]:
images = {}
with open(cat_jpg, 'rb') as file: images[cat_jpg] = file.read()
with open(dog_jpg, 'rb') as file: images[dog_jpg] = file.read()
with open(ImageNetLabels, 'r') as file: class_id_to_label = file.read().splitlines()

### Query endpoint that you have created

In [17]:
import json

def query_endpoint(img):
    endpoint_name = 'jumpstart-dft-et-poc-inception-tf-imgnet-v1'
    client = boto3.client('runtime.sagemaker')
    response = client.invoke_endpoint(EndpointName=endpoint_name, ContentType='application/x-image', Body=img)
    model_predictions = json.loads(response['Body'].read())['predictions'][0]
    return model_predictions

for filename, img in images.items():
    model_predictions = query_endpoint(img)  
    top5_prediction_ids = sorted(range(len(model_predictions)), key=lambda index: model_predictions[index], reverse=True)[:5]
    top5_class_labels = ", ".join([class_id_to_label[id] for id in top5_prediction_ids])
    display(HTML(f'<img src={filename} alt={filename} align="left" style="width: 250px;"/>' 
                 f'<figcaption>Top-5 model predictions are: {top5_class_labels}</figcaption>'))

## Test Sunflowers

In [18]:
filename = './Sunflower-seed-pic-Biofuels-12-8-768x512.jpeg'

with open(filename, 'rb') as file: 
    img = file.read()


model_predictions = query_endpoint(img)  
top5_prediction_ids = sorted(range(len(model_predictions)), key=lambda index: model_predictions[index], reverse=True)[:5]
top5_class_labels = ", ".join([class_id_to_label[id] for id in top5_prediction_ids])
display(HTML(f'<img src={filename} alt={filename} align="left" style="width: 250px;"/>' 
             f'<figcaption>Top-5 model predictions are: {top5_class_labels}</figcaption>'))


## Test Sunflowers on new model (Transfer Learning on TF-Flowers Dataset)

### Download the Model and Extract the Class Name Index

In [6]:
!aws s3 cp s3://sagemaker-us-east-1-123456789012/smjs-d-et-poc-inception-tf-tfflower-xferl-v2-20210428-171350/output/model.tar.gz . 

download: s3://sagemaker-us-east-1-123456789012/smjs-d-et-poc-inception-tf-tfflower-xferl-v2-20210428-171350/output/model.tar.gz to ./model.tar.gz


In [7]:
!tar zxvf model.tar.gz class_label_to_prediction_index.json

class_label_to_prediction_index.json


### Test Sunflowers again

In [22]:
import json

with open('class_label_to_prediction_index.json') as f:
    label_to_class_id = json.load(f)
    class_id_to_label = {v:k for k, v in label_to_class_id.items()}

In [23]:
def query_fine_tuned_endpoint(img):
    endpoint_name = 'jumpstart-ftd-et-poc-inception-tf-tfflower-xferl-v2'
    client = boto3.client('runtime.sagemaker')
    response = client.invoke_endpoint(EndpointName=endpoint_name, ContentType='application/x-image', Body=img)
    model_predictions = json.loads(response['Body'].read())['predictions'][0]
    return model_predictions

In [24]:
model_predictions = query_fine_tuned_endpoint(img)  
top5_prediction_ids = sorted(range(len(model_predictions)), key=lambda index: model_predictions[index], reverse=True)[:5]
top5_class_labels = ", ".join([class_id_to_label[id] for id in top5_prediction_ids])
display(HTML(f'<img src={filename} alt={filename} align="left" style="width: 250px;"/>' 
             f'<figcaption>Top-5 model predictions are: {top5_class_labels}</figcaption>'))

## ET POC SKU Test

In [26]:
! aws s3 cp s3://sagemaker-us-east-1-123456789012/smjs-c-et-poc-inception-skudataset-xferl-v3-20210428-202031/output/model.tar.gz .

download: s3://sagemaker-us-east-1-123456789012/smjs-c-et-poc-inception-skudataset-xferl-v3-20210428-202031/output/model.tar.gz to ./model.tar.gz


In [27]:
!tar zxvf model.tar.gz class_label_to_prediction_index.json

class_label_to_prediction_index.json


### Test

In [42]:
import json

with open('class_label_to_prediction_index.json') as f:
    label_to_class_id = json.load(f)
    class_id_to_label = {v:k for k, v in label_to_class_id.items()}

In [43]:
label_to_class_id

{'BenimO': 0,
 'BrowniGoldVisne': 1,
 'FormKepekli': 2,
 'FormPatlak': 3,
 'Gong': 4,
 'Gonlubol': 5,
 'PopkekLimonlu': 6,
 'PopkekMuzlu': 7,
 'PufKakaolu': 8}

In [44]:
class_id_to_label

{0: 'BenimO',
 1: 'BrowniGoldVisne',
 2: 'FormKepekli',
 3: 'FormPatlak',
 4: 'Gong',
 5: 'Gonlubol',
 6: 'PopkekLimonlu',
 7: 'PopkekMuzlu',
 8: 'PufKakaolu'}

In [45]:
def query_et_sku_endpoint(img):
    endpoint_name = 'jumpstart-ftc-et-poc-inception-skudataset-xferl-v3'
    client = boto3.client('runtime.sagemaker')
    response = client.invoke_endpoint(EndpointName=endpoint_name, ContentType='application/x-image', Body=img)
    model_predictions = json.loads(response['Body'].read())['predictions'][0]
    return model_predictions

In [46]:
filename = './eti_form_kepekli.jpg'

with open(filename, 'rb') as file: 
    img = file.read()


In [50]:
model_predictions = query_et_sku_endpoint(img)  
top5_prediction_ids = sorted(range(len(model_predictions)), key=lambda index: model_predictions[index], reverse=True)[:5]
top5_class_labels = ", ".join([class_id_to_label[id] for id in top5_prediction_ids])
display(HTML(f'<img src={filename} alt={filename} align="left" style="width: 250px;"/>' 
             f'<figcaption>Top-5 model predictions are: {top5_class_labels}</figcaption>'))
print(f'Top Class Labels: {top5_class_labels}')

Top Class Labels: FormKepekli, BenimO, Gong, PopkekLimonlu, PopkekMuzlu
